In [12]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

%matplotlib tk

In [95]:
###データ読み込み
###学習データ
dataframe = pandas.read_csv('2lstm_testdata_kubo_up.csv', usecols=[0,1], engine='python')
###テストデータ
dataframe2 = pandas.read_csv('2lstm_testdata_okugawa_30.csv', usecols=[0,1], engine='python')

#plt.figure(figsize=(80,10))
#plt.plot(dataframe2)
#plt.show()
#print(dataframe.head())

In [96]:
###np.arrayに変換し、キャスト
dataset = dataframe.values
dataset2 = dataframe2.values
dataset = dataset.astype('float32')
dataset2 = dataset2.astype('float32')

###データを標準化
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
dataset2 = scaler.fit_transform(dataset2)

#print(dataset)
#plt.figure(figsize=(80,10))
#plt.plot(dataset)
#plt.show()

###学習データ切り分け
#train_size = int(len(dataset) * 0.67)
#test_size = len(dataset) - train_size
#train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
#print(dataset.shape[1])

###学習とテストで切り分ける場合
print(len(dataset))
print(len(dataset2))
train_size = len(dataset)
test_size = len(dataset2)
train, test = dataset[:,:], dataset2[:,:]

2662
2678


In [97]:
###データを入力データと出力データにわける関数
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        xset = []
        for j in range(dataset.shape[1]):
            a = dataset[i:(i+look_back), j]
            xset.append(a)
        dataY.append(dataset[i + look_back, 1])      
        dataX.append(xset)
    return numpy.array(dataX), numpy.array(dataY)

### X=t, Y=t+1に切り分け
look_back = 300
trainX, trainY = create_dataset(train, look_back)#学習データ
testX, testY = create_dataset(test, look_back)#教師データ

print(trainX.shape[0])
print(trainX.shape[1])
print(trainX.shape[2])
print(trainX)

### 3次元配列に変換[samples, time steps(number of variables), features] 
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], testX.shape[2]))
print(trainX)

2361
2
300
[[[0.9035064  0.9353831  0.9629561  ... 0.7220943  0.686618   0.651718  ]
  [0.75275004 0.75998455 0.77165246 ... 0.8883687  0.91346806 0.93419963]]

 [[0.9353831  0.9629561  0.9919632  ... 0.686618   0.651718   0.6200747 ]
  [0.75998455 0.77165246 0.78698814 ... 0.91346806 0.93419963 0.9407447 ]]

 [[0.9629561  0.9919632  0.978621   ... 0.651718   0.6200747  0.5785614 ]
  [0.77165246 0.78698814 0.8006517  ... 0.93419963 0.9407447  0.935231  ]]

 ...

 [[0.01357178 0.0452641  0.07487026 ... 0.24413124 0.27334532 0.31328028]
  [0.03523774 0.03264837 0.02554137 ... 0.08106816 0.10579465 0.13516726]]

 [[0.0452641  0.07487026 0.10391176 ... 0.27334532 0.31328028 0.33944348]
  [0.03264837 0.02554137 0.0164409  ... 0.10579465 0.13516726 0.16455822]]

 [[0.07487026 0.10391176 0.12939262 ... 0.31328028 0.33944348 0.36643806]
  [0.02554137 0.0164409  0.005068   ... 0.13516726 0.16455822 0.19299914]]]
[[[0.9035064  0.9353831  0.9629561  ... 0.7220943  0.686618   0.651718  ]
  [0.7527

In [68]:
### LSTMネットワーク作成
model = Sequential()
model.add(LSTM(100, input_shape=(testX.shape[1], look_back)))	#input_shape：変数数、遡る時間数
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
early_stopping = EarlyStopping(monitor='val_loss', mode='auto', patience=20)#過学習防止
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=1, callbacks=[early_stopping])

Epoch 1/100
2361/2361 [==============================] - 5s 2ms/step - loss: 0.0175
Epoch 2/100
  85/2361 [>.............................] - ETA: 4s - loss: 0.0027  

C:\Users\hai\Anaconda3\envs\textgen\lib\site-packages\keras\callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


2361/2361 [==============================] - 4s 2ms/step - loss: 0.0046
Epoch 3/100
2361/2361 [==============================] - 4s 2ms/step - loss: 0.0040
Epoch 4/100
2361/2361 [==============================] - 4s 2ms/step - loss: 0.0027
Epoch 5/100
2361/2361 [==============================] - 4s 2ms/step - loss: 0.0024
Epoch 6/100
2361/2361 [==============================] - 4s 2ms/step - loss: 0.0019
Epoch 7/100
2361/2361 [==============================] - 4s 2ms/step - loss: 0.0018
Epoch 8/100
2361/2361 [==============================] - 4s 2ms/step - loss: 0.0015
Epoch 9/100
2361/2361 [==============================] - 4s 2ms/step - loss: 0.0014
Epoch 10/100
2361/2361 [==============================] - 5s 2ms/step - loss: 0.0013
Epoch 11/100
2361/2361 [==============================] - 5s 2ms/step - loss: 0.0012
Epoch 12/100
2361/2361 [==============================] - 5s 2ms/step - loss: 0.0011
Epoch 13/100
2361/2361 [==============================] - 5s 2ms/step - loss: 0.0010


2361/2361 [==============================] - 5s 2ms/step - loss: 4.0645e-04
Epoch 95/100
2361/2361 [==============================] - 5s 2ms/step - loss: 4.1692e-04
Epoch 96/100
2361/2361 [==============================] - 5s 2ms/step - loss: 4.1857e-04
Epoch 97/100
2361/2361 [==============================] - 5s 2ms/step - loss: 4.4908e-04
Epoch 98/100
2361/2361 [==============================] - 5s 2ms/step - loss: 4.5660e-04
Epoch 99/100
2361/2361 [==============================] - 5s 2ms/step - loss: 4.0503e-04
Epoch 100/100
2361/2361 [==============================] - 5s 2ms/step - loss: 4.4951e-04


In [98]:
### 作成したモデルで予測
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

###予想データ、Yデータを標準化前の数字に戻す
#scalerが元のデータセットと同じ形でないと受け付けられないため存在していたカラムの数だけ値を0で埋める
print(dataset.shape[1])
pad_col = numpy.zeros(dataset.shape[1]-1)
def pad_array(val):
    return numpy.array([numpy.insert(pad_col, 0, x) for x in val])
    
trainPredict = scaler.inverse_transform(pad_array(trainPredict))
trainY = scaler.inverse_transform(pad_array(trainY))
testPredict = scaler.inverse_transform(pad_array(testPredict))
testY = scaler.inverse_transform(pad_array(testY))

###標準偏差
trainScore = math.sqrt(mean_squared_error(trainY[:,0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[:,0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#print(len(testPredict))

2
Train Score: 1.49 RMSE
Test Score: 8.29 RMSE


In [100]:
print(testY[:,0])
print(testPredict[:,0])
###プロット用にデータ修正
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

###プロット用にデータ修正
testPredictPlot = numpy.empty_like(dataset2)
testPredictPlot[:, :] = numpy.nan
#testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
#testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset2)-1, :] = testPredict

###グラフ描画
#plt.plot(scaler.inverse_transform(dataset2))
#plt.plot(trainPredictPlot)
plt.plot(testPredict)
plt.plot(testY)

plt.show()

[23.98875315 25.73578793 27.42563931 ... 61.58350904 61.56975522
 61.84404362]
[29.72813315 32.0560174  34.33585443 ... 54.24420274 55.02232783
 55.56741524]
